In [223]:
from ipywidgets import interact
import numpy as np
from bokeh.layouts import row, column, layout
from bokeh.models.widgets import Panel, Tabs

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d
import time
import bokeh
from bokeh.palettes import Dark2_5 as palette
# itertools handles the cycling
import itertools  

colors = itertools.cycle(palette)
from bokeh.models.widgets import Button, Toggle
from bokeh.models.widgets import RadioButtonGroup, DataTable, DateFormatter, TableColumn

output_notebook()

Loading BokehJS ...

In [7]:
t0 = time.time()

import zmq
from silayer.raw2hdf import DataPacket

def loop(data_socket):
    while True:
        data = data_socket.recv() ## This should block until data shows up
        dp = DataPacket(data)
        ## Then do whatever...
        
def main(host="si-layer.local", data_port=9998):
    ctx = zmq.Context()
    data_addr = f"tcp://{host}:{data_port}"
    data_socket = ctx.socket(zmq.SUB)
    data_socket.setsockopt_string(zmq.SUBSCRIBE, "")
    data_socket.connect(data_addr)
    
    return data_socket

socket = main("10.10.0.11")

In [269]:
def make_channel_display(datasource):
    channel_display = figure(title='channel', tools='', 
           plot_height=350, plot_width=800,
           background_fill_color="#fafafa", )
    channel_display.xaxis.axis_label = "channel"
    channel_display.yaxis.axis_label = "adc counts (LSB)"
    #p.y_range = Range1d(0, 400)

    steps = channel_display.step('channels', 'values', source=datasource)
    
    return channel_display

def make_channel_timestream(datasource, show_ch = [0, 1, 2]):
    channel_stream = figure(title='ASIC_<x>', tools='', 
           plot_height=350, plot_width=400,
           background_fill_color="#fafafa", )

    channel_stream.xaxis.axis_label = "time (s)"
    channel_stream.yaxis.axis_label = "adc counts (LSB)"

    for ch in show_ch:
        color=next(colors)
        channel_stream.line('time', f'ch{ch:02d}', source=datasource, color=color)
        channel_stream.circle('time', f'ch{ch:02d}', source=datasource, legend_label=f'ch{ch:02d}', color=color)

    return channel_stream

def make_channel_binner(datasource, show_ch = [0, 1, 2]):
    hist_fig = figure(title='ASIC_<x>', #tools='', 
                      plot_height=350, plot_width=400,
                      background_fill_color="#fafafa", )

    hist_fig.xaxis.axis_label = "adc counts (LSB)"

    rate_fig = figure(title='event rate', tools='',
                      plot_height=350, plot_width=400,
                      background_fill_color="#fafafa", )                  


    #print("Making histograms for channels: ", show_ch)
    for ch in show_ch:
        hist_fig.quad(top=f'ch{ch:02d}', left='left', right='right', bottom='bottom', 
                      source=datasource, color=next(colors), legend_label=f'ch{ch:02d}')
                
    return hist_fig
        


n_ch = 32
channel_ds = ColumnDataSource(data={'values': np.zeros(n_ch), 'channels': np.array(range(n_ch))})
                
timeseries = {'time': np.zeros(0)}
for ch in range(32):
    timeseries[f'ch{ch:02d}'] = []

#print(timeseries.keys())
timestream_ds = ColumnDataSource(data=timeseries)


ch_display = make_channel_display(channel_ds)
ts_display = make_channel_timestream(timestream_ds)

#hist_display = make_channel_binner(hist_ds)

In [291]:
def calc_mean(bin_centers, binned_data):
    return np.sum(bin_centers*binned_data) / np.sum(binned_data)
def calc_var(bin_centers, binned_data):
    mean = calc_mean(bin_centers, binned_data)
    return np.sum(binned_data*(bin_centers - mean)**2) / (np.sum(binned_data))


ctr = np.array([-1, 0, 2])
cts = np.array([5, 1, 6])

print(calc_mean(ctr, cts))
np.sqrt(calc_var(ctr, cts))

calc_mean(np.array([ctr, ctr]), np.array([cts, cts]))

0.5833333333333334


0.5833333333333334

In [271]:
bin_edges = np.array(range(1025)) - 0.5
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
binned_data = np.zeros(len(bin_centers))

hdict = {}
for ch in range(n_ch):
    hdict[f'ch{ch:02d}'] = np.zeros(len(bin_centers))

hdict['bin_centers'] = bin_centers
hdict['left'] = bin_edges[0:-1]
hdict['right'] = bin_edges[1::]
hdict['bottom'] = np.zeros(len(bin_centers))

binned_data_ds = ColumnDataSource(hdict)

hist_display = make_channel_binner(binned_data_ds)
hist_display_list = [make_channel_binner(binned_data_ds, [i]) for i in range(n_ch)]
show(Tabs(tabs=[Panel(child=hist_display_list[i], title=f'ch{i:02d}') for i in range(3)]))

In [272]:
def get_layout():
    """Create layout and app elements."""

    data_scr = ColumnDataSource({"x":np.random.normal(size=(10,)),
                                 "y":np.random.normal(size=(10,))})
    columns = [TableColumn(field="x", title="First"),
               TableColumn(field="y", title="Second")]

    full_table = DataTable(columns=columns, source=data_scr)

    mask = [True for el in data_scr.data["y"]]
    data_view = CDSView(source=data_scr, filters=[BooleanFilter(mask)])
    filtered_table = DataTable(columns=columns, source=data_scr, view=data_view)

    lt = layout([[full_table], [filtered_table]])
    return data_scr, filtered_table, lt

In [325]:
stats_ds = ColumnDataSource({'mean': np.zeros(n_ch), 'sigma': np.zeros(n_ch), 'N': np.zeros(n_ch)})

hfmt = bokeh.models.NumberFormatter(format="0.00", text_align='center')     
columns = [TableColumn(field="mean", title="μ", formatter=hfmt), 
           TableColumn(field="sigma", title="σ", formatter=hfmt), 
           TableColumn(field="N", title="N", formatter=hfmt)]
                        
data_table = DataTable(source=stats_ds, columns=columns, width=400, height=280, 
                       #index_position=None, 
                       sortable=False,
                      editable=False, selectable=False, reorderable=False,fit_columns=True
                      )
           
show(data_table)


In [326]:
#t = #show(column(ch_display, row(ts_display, ts_display)), notebook_handle=True)
channel_tabs = Tabs(tabs=[Panel(child=hist_display_list[i], title=f'ch{i:02d}') for i in range(3)])
t = show(column(ch_display, row(hist_display, ts_display), data_table),
                                
         notebook_handle=True)
#ts_handle = show(channel_stream, notebook_handle=True)

In [329]:
#channel_ds = ColumnDataSource(data={'pedestals': np.zeros(n_ch), 'channels': np.array(range(n_ch))})
channels = np.array(range(n_ch))

event_counter = 5000
for i in range(100):
    event_counter += 1
    
    ped_data = 250 + np.random.normal(scale=5, size=n_ch)
    channel_ds.data['values'] = ped_data
    timeseries = {'time': [event_counter]}
    patches = {}
    
    for ch in range(n_ch):
        ch_name = f'ch{ch:02d}'
        
        timeseries[ch_name] = [ped_data[ch]]      
        
        bin_number = int(np.histogram(ped_data[ch], bins=bin_edges)[0].argmax())   
        old_bin_content = binned_data_ds.data[ch_name][bin_number]
        patches[f'ch{ch:02d}'] = [(bin_number, old_bin_content+1)]

    binned_data_ds.patch(patches)
    timestream_ds.stream(timeseries, rollover=100)    
    
    
    #Downscale updating of rolling average. 
    if event_counter %10 == 0:
        stats_patches = {}

        stats_patches['mean'] = []
        stats_patches['sigma'] = []
        stats_patches['N'] = []
        for ch in range(3):
            ch_name = f'ch{ch:02d}'        
            stats_patches['mean'] += [(ch, calc_mean(bin_centers, binned_data_ds.data[ch_name]))]   
            stats_patches['sigma'] += [(ch, np.sqrt(calc_var(bin_centers, binned_data_ds.data[ch_name])))]   
            stats_patches['N'] += [(ch, np.sum(binned_data_ds.data[ch_name]))]   

        
        stats_ds.patch(stats_patches)
    #binned_data_ds.patch(patches)
    
    time.sleep(np.abs(np.random.normal(0.05, scale=0.01)))
    
    push_notebook(t)
    


In [ ]:
#for i in range(100):
i = 0
n_received = 0
while True:
    data = socket.recv()
    n_received += 1
    i += 1
    if i % 200 == 0 and i != 0:
        i = 0
        dp = DataPacket(data)
        ap = dp.asic_packets[0]

        steps.data_source.data['values']  = ap.data

        
        datastream.stream({'time': [dp.event_counter], 
                           'adc0':[ap.data[0]], 
                           'adc1':[ap.data[1]]}, 
                           rollover=1000)    
        #datastream.stream({'time': np.zeros(32) + dp.event_counter, 
        #                   'adc': ap.data}, rollover=100)    
    
    push_notebook(t)